In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/paths"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import * 
qualifying_schema = StructType([StructField('qualifyId', IntegerType(), True),
                               StructField('raceId', IntegerType(), True),
                               StructField('driverId', IntegerType(), True),
                               StructField('constructorId', IntegerType(), True),
                               StructField('number', IntegerType(), True),
                               StructField('position', IntegerType(), True),
                               StructField('q1', StringType(), True),
                               StructField('q2', StringType(), True),
                               StructField('q3', StringType(), True)])

In [0]:
qualify_df = spark.read.schema(qualifying_schema).option("multiline", True).json(f"{raw_container_folder_path}/{v_file_date}/qualifying")

In [0]:
from pyspark.sql.functions import current_timestamp, lit
qualify_final_df = qualify_df.withColumnRenamed("qualifyId", "qualify_id")\
    .withColumnRenamed("driverId", "driver_id")\
    .withColumnRenamed("raceId", "race_id")\
    .withColumnRenamed("constructorId", "constructor_id")\
    .withColumn("ingested_date", current_timestamp())\
    .withColumn("data_source", lit(v_data_source))

In [0]:
#qualify_final_df.write.mode("overwrite").parquet(f"{processed_container_folder_path}/qualifying")

#qualify_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed_db.qualifying")

#load_incremental_data(qualify_final_df, "race_id","f1_processed_db", "qualifying")

merge_condition = "target_tab.qualify_id = source_tab.qualify_id AND target_tab.race_id = source_tab.race_id"
merge_incremental_data( qualify_final_df, processed_container_folder_path, merge_condition, "race_id", "f1_processed_db", "qualifying")

In [0]:
dbutils.notebook.exit("success")